In this notebook I do simple EDA, ending with plotting some of the training data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import patches, patheffects
import imageio

In [ ]:
def read_image_locs(dirname='../input/siim-covid19-detection/train'):
    collect = []
    for dirname, _, filenames in os.walk(dirname):
        for filename in filenames:
            study, series = dirname.split("/")[-2:]
            image = filename.replace(".dcm", "")
            collect.append((study, series, image))
    locs = pd.DataFrame(collect, columns=("study", "series", "image"))
    return locs

train_image_locs = read_image_locs()

In [ ]:
image_level = pd.read_csv("../input/siim-covid19-detection/train_image_level.csv")
study_level = pd.read_csv("../input/siim-covid19-detection/train_study_level.csv")

In [ ]:
image_level.head()

In [ ]:
study_level.head()

Here I merge the study and image dataframes to get study id and labels associated with each image

In [ ]:
study_level_ = study_level.copy()
study_level_.id = study_level_.id.str.replace("_study", "")
merged = image_level.merge(study_level_, left_on="StudyInstanceUID", right_on="id", suffixes=("_image", "_study"))
merged = merged.rename({'Negative for Pneumonia': 'Negative', 'Typical Appearance': 'Typical', 'Indeterminate Appearance': 'Indeterminate', 'Atypical Appearance': 'Atypical'}, 
              axis="columns")
merged.head()

Here I check that the merged dataframe contains the same number of rows as the image dataframe

In [ ]:
merged.shape, image_level.shape, study_level.shape

Here I check that the last four columns are indeed one hot encoded, i.e. that exactly one of these equals one, while the three equals zero for each row

In [ ]:
(merged[['Negative', 'Typical', 'Indeterminate', 'Atypical']].sum(axis="columns") != 1).sum()

# Plot X-ray

In [ ]:
def get_boxes_and_labels(image_info):
    if pd.isna(image_info.boxes):
        boxes, labels = [], []
    else:
        boxes = eval(image_info.boxes)
        label = image_info[['Negative', 'Typical', 'Indeterminate', 'Atypical']].T
        label = label[label.astype("bool")].index[0]
        labels = len(boxes)*[label]
    return boxes, labels
    
def draw_boxes(ax, image_info, color="tab:orange"):
    for box, label in zip(*get_boxes_and_labels(image_info)):
        rect = patches.Rectangle((box["x"], box["y"]), box["width"], box["width"], fill=False, edgecolor=color)
        ax.add_patch(rect)
        text = ax.text(box["x"], box["y"], label, color=color, fontsize=20)
        text.set_path_effects([patheffects.Stroke(linewidth=3, foreground='black'),
                   patheffects.Normal()]) 
def plot_xray(idx, ax):
    image_info = merged.iloc[idx]
    image_loc = train_image_locs[train_image_locs["study"] == image_info["StudyInstanceUID"]]
    image_file = '../input/siim-covid19-detection/train/' + "/".join(image_loc.values[0,:]) + ".dcm"
    im = imageio.imread(image_file)
    ax.imshow(im, cmap="gray")
    ax.axis("off")
    
    draw_boxes(ax, image_info)

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(20, 20))
idxs = [i for i in range(axes.size)]
print(type(axes))
for idx, ax in zip(idxs, axes.flat):
    plot_xray(idx, ax)

The brightness varies a lot between each image, so some form of brightness equilization should be done.